In [1]:
import tensorflow as tf
tf.__version__

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.0'

In [2]:
from tensorflow.contrib import keras

In [3]:
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
import os

np.random.seed(1337)

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys
import datetime

now = datetime.datetime.now

In [5]:
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [7]:
img_rows, img_cols = IMG_WIDTH, IMG_HEIGHT

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)


In [8]:
shape_ord

(224, 224, 3)

In [9]:
from keras.applications import vgg16
from keras.layers import Input

# build the VGG16 network with ImageNet weights
vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)))
print('Model loaded.')

Model loaded.


In [10]:
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization

In [11]:
for l in vgg16.layers:
    l.trainable = False

In [12]:
x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = Dense(2048, activation='relu', name='ft_fc12')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(4, activation = 'softmax')(x)

In [13]:
from keras.models import Model

model = Model(inputs=vgg16.input, outputs=predictions)

#compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

In [16]:
train_generator = train_datagen.flow_from_directory(
        '../data/train',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 3781 images belonging to 4 classes.


In [17]:
validation_datagen = ImageDataGenerator()

In [18]:
validation_generator = validation_datagen.flow_from_directory(
        '../data/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 1261 images belonging to 4 classes.


In [19]:
test_datagen = ImageDataGenerator()

In [20]:
test_generator = test_datagen.flow_from_directory(
        '../data/test',
        target_size=(224,224),
        batch_size=32,
        class_mode='categorical')

Found 561 images belonging to 4 classes.


In [21]:
import PIL
from PIL import Image

In [22]:
import sys
print(sys.executable)

/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=118,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=39,
        workers=-1,
        use_multiprocessing=True)